In [1]:
!pip install -q mediapipe


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 8.8 MB/s eta 0:00:00


In [2]:
!wget -O classifier.tflite -q https://storage.googleapis.com/mediapipe-models/text_classifier/bert_classifier/float32/1/bert_classifier.tflite


In [7]:
# Define the input text that you wants the model to classify.
INPUT_TEXT = "I'm looking forward to what will come next."
INPUT_TEXT1 = "The meeting is scheduled for tomorrow."

In [8]:
# STEP 1: Import the necessary modules.
from mediapipe.tasks import python
from mediapipe.tasks.python import text

# STEP 2: Create an TextClassifier object.
base_options = python.BaseOptions(model_asset_path="classifier.tflite")
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# STEP 3: Classify the input text.
classification_result = classifier.classify(INPUT_TEXT1)

# STEP 4: Process the classification result. In this case, print out the most likely category.
top_category = classification_result.classifications[0].categories[0]
print(f'{top_category.category_name} ({top_category.score:.2f})')

positive (0.56)


# Streamlit APP

In [9]:
!pip install -q streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.7 MB/s eta 0:00:00


In [10]:
import os
from threading import Thread
from pyngrok import ngrok


In [11]:

from google.colab import userdata

# Retrieve the Ngrok API key securely
ngrok_api_key = userdata.get('NGROK_API')

if ngrok_api_key:
    # Authenticate Ngrok with the API key
    ngrok.set_auth_token(ngrok_api_key)
    print("Ngrok authenticated successfully!")
else:
    print("Ngrok API key not found. Please add it in the Colab settings.")

Ngrok authenticated successfully!


In [12]:
# Add your ngrok token here
ngrok.set_auth_token(ngrok_api_key)

In [14]:
%%writefile app.py
import streamlit as st
from mediapipe.tasks import python
from mediapipe.tasks.python import text

# Initialize the classifier with the model
base_options = python.BaseOptions(model_asset_path="classifier.tflite")
options = text.TextClassifierOptions(base_options=base_options)
classifier = text.TextClassifier.create_from_options(options)

# Streamlit app
st.title('Text Sentiment Classifier')

# User input
user_input = st.text_input("Enter a sentence:")

if user_input:
    # Classify input text
    classification_result = classifier.classify(user_input)
    top_category = classification_result.classifications[0].categories[0]

    # Display result with percentage
    sentiment = top_category.category_name
    confidence = top_category.score

    # Set color based on sentiment
    if sentiment.lower() == 'positive':
        color = "green"
    else:
        color = "red"

    # Display the result
    st.markdown(f"### Sentiment: {sentiment.capitalize()} ({confidence * 100:.2f}%)")
    st.markdown(f'<div style="background-color:{color}; padding: 10px; border-radius: 5px; color: white; font-size: 20px;">{sentiment.capitalize()} - {confidence * 100:.2f}%</div>', unsafe_allow_html=True)


Overwriting app.py


In [15]:
def run_streamlit():
    # Change the port if 8501 is already in use or if you prefer another port
    os.system('streamlit run /content/app.py --server.port 8501')

In [16]:
# Start a thread to run the Streamlit app
thread = Thread(target=run_streamlit)
thread.start()


In [17]:
# Open a tunnel to the streamlit port 8501
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Your Streamlit app is live at:', public_url)

Your Streamlit app is live at: NgrokTunnel: "https://14b0-35-238-23-50.ngrok-free.app" -> "http://localhost:8501"


In [18]:
ngrok.kill()